<a href="https://colab.research.google.com/github/leticyadanielly/projeto1/blob/main/PROJETO_RELOGIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorflow
!pip install scikit-learn
!pip install numpy
!pip install Pillow

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.neighbors import NearestNeighbors
from PIL import Image
import os
import glob

# --- 1. Configuração e Carregamento do Modelo Pré-treinado ---
# Usamos o VGG16, mas você pode experimentar com ResNet50, EfficientNet, etc.
# 'weights=imagenet' garante que o modelo use os pesos pré-treinados.
# 'include_top=False' remove a camada de classificação final.
# 'pooling=avg' nos dá um vetor de características único e de tamanho fixo.
model = VGG16(weights='imagenet', include_top=False, pooling='avg')

print("Modelo VGG16 carregado com sucesso para extração de características.")

# --- 2. Preparação dos Dados (Simulação) ---
# Vamos simular um diretório de imagens.
# Crie uma pasta 'dataset' e coloque algumas imagens lá.
# Exemplo: 'dataset/relogio1.jpg', 'dataset/relogio2.png', etc.
dataset_path = 'dataset'
if not os.path.exists(dataset_path):
    print(f"Diretório '{dataset_path}' não encontrado. Criando um diretório de exemplo.")
    os.makedirs(dataset_path)


image_files = glob.glob(os.path.join(dataset_path, '*'))
if not image_files:
    print(f"--------------------------------------------------------------------")
    print(f"ATENÇÃO: A pasta '{dataset_path}' está vazia.")
    print(f"Por favor, adicione imagens dos produtos (e.g., relógios) na pasta '{dataset_path}'.")
    print(f"Após adicionar as imagens, execute esta célula novamente.")
    print(f"--------------------------------------------------------------------")
    # Return from the cell to stop execution without exiting the kernel
    from google.colab.output import eval_js
    eval_js('google.colab.kernel.restart()')
    # Added a kernel restart to make sure the user sees the message and re-runs the cell
    # after adding images. This is a workaround because a simple return doesn't stop execution
    # in the middle of a cell in the same way as a direct `exit()`.
    exit()


print(f"Encontradas {len(image_files)} imagens no diretório.")

# --- 3. Função para Extrair Características de uma Única Imagem ---
def extract_features(img_path, model):
    """
    Carrega, pré-processa e extrai o vetor de características de uma imagem.
    """
    try:
        img = Image.open(img_path).resize((224, 224)) # VGG16 espera imagens 224x224
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array) # Pré-processamento específico para o VGG16
        features = model.predict(img_array, verbose=0)
        return features.flatten() # Retorna o vetor de características como um array 1D
    except Exception as e:
        print(f"Erro ao processar a imagem {img_path}: {e}")
        return None

# --- 4. Extrair Características para TODO o Conjunto de Dados ---
features_list = []
image_names = []

print("Extraindo características das imagens do conjunto de dados...")
for img_path in image_files:
    features = extract_features(img_path, model)
    if features is not None:
        features_list.append(features)
        image_names.append(os.path.basename(img_path))

features_matrix = np.array(features_list)
print("Extração de características concluída.")
print(f"Matriz de características gerada com shape: {features_matrix.shape}")


# --- 5. Construção do Algoritmo de Busca de Vizinhos (KNN) ---
# Usamos NearestNeighbors do scikit-learn, que é eficiente para encontrar os vetores mais próximos.
# 'metric=cosine' é uma boa métrica para similaridade de vetores de características.
# Ensure n_neighbors does not exceed the number of samples
n_neighbors = min(5, len(image_files)) # Quantos produtos similares queremos recomendar
if n_neighbors == 0:
    print("Não há imagens para treinar o modelo KNN. Por favor, adicione imagens ao diretório 'dataset'.")
    # Another check and early exit if no images were processed
    exit()

nn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
nn_model.fit(features_matrix)

print(f"Modelo de busca de vizinhos (KNN) treinado para encontrar {n_neighbors} vizinhos.")

# --- 6. Função de Recomendação (Execução) ---
def recommend_similar_products(query_img_path, model, nn_model, image_names):
    """
    Recebe o caminho de uma imagem de consulta e retorna os produtos mais similares.
    """
    print(f"\nBuscando produtos similares para a imagem: {query_img_path}...")

    # Extrair características da imagem de consulta
    query_features = extract_features(query_img_path, model)
    if query_features is None:
        return []

    # Encontrar os vizinhos mais próximos no banco de dados
    distances, indices = nn_model.kneighbors([query_features])

    # Retornar os nomes dos arquivos das imagens similares
    similar_images = [image_names[i] for i in indices[0]]

    # O primeiro resultado é a própria imagem de consulta, então a removemos
    # ou ajustamos a busca para n+1 e removemos o primeiro.
    similar_images_filtered = [img for img in similar_images if img != os.path.basename(query_img_path)]

    print("Recomendação concluída!")
    return similar_images_filtered

# --- 7. Exemplo de Uso ---
# Substitua 'dataset/imagem_de_consulta.jpg' pelo caminho de uma de suas imagens.
if image_files: # Check if there are any images to use as a query
  query_image_path = image_files[0]  # Usamos a primeira imagem do nosso dataset como exemplo
  recommendations = recommend_similar_products(query_image_path, model, nn_model, image_names)

  print("Produtos recomendados (por nome de arquivo):")
  for rec in recommendations:
      print(f"- {rec}")
else:
  print("Não há imagens no diretório 'dataset' para executar o exemplo de uso.")

Modelo VGG16 carregado com sucesso para extração de características.
Encontradas 3 imagens no diretório.
Extraindo características das imagens do conjunto de dados...
Extração de características concluída.
Matriz de características gerada com shape: (3, 512)
Modelo de busca de vizinhos (KNN) treinado para encontrar 3 vizinhos.

Buscando produtos similares para a imagem: dataset/download.jpg...
Recomendação concluída!
Produtos recomendados (por nome de arquivo):
- download (2).jpg
- download (1).jpg
